给你一个二维数组 edges 表示一个 n 个点的无向图，其中 edges[i] = [ui, vi, lengthi] 表示节点 ui 和节点 vi 之间有一条需要 lengthi 单位时间通过的无向边。

同时给你一个数组 disappear ，其中 disappear[i] 表示节点 i 从图中消失的时间点，在那一刻及以后，你无法再访问这个节点。

注意，图有可能一开始是不连通的，两个节点之间也可能有多条边。

请你返回数组 answer ，answer[i] 表示从节点 0 到节点 i 需要的 最少 单位时间。如果从节点 0 出发 无法 到达节点 i ，那么 answer[i] 为 -1 。



对于本题，answer 几乎就是 dis 数组。只需要在 Dijkstra 算法的过程中，添加一个判断：

在更新最短路之前，如果最短路长度 ≥disappear[i]，说明无法及时到达节点 i，不更新。

问：我是这样做的，先不管 disappear，用 Dijkstra 算法模板求出 dis 数组，然后把其中 dis[i]≥disappear[i] 的 dis[i] 改成 −1。这个做法是否正确？

答：这个做法是错的。考虑这样的数据：离节点 0 近的节点 x，其 disappear[x] 很小；离节点 0 远的节点 y，其 disappear[y] 很大。由于 disappear[x] 很小，我们无法通过节点 x，所以远处的节点 y 实际上也无法到达。但错误做法会因为 dis[y]<disappear[y]，误认为节点 y 可以到达，最终返回错误的答案。

In [ ]:
from heapq import heappop, heappush
from typing import List


class Solution:
    def minimumTime(
        self, n: int, edges: List[List[int]], disappear: List[int]
    ) -> List[int]:
        g = [[] for _ in range(n)]  # 稀疏图用邻接表
        for x, y, wt in edges:
            g[x].append((y, wt))
            g[y].append((x, wt))

        dis = [-1] * n
        dis[0] = 0
        h = [(0, 0)]
        while h:
            dx, x = heappop(h)
            if dx > dis[x]:
                continue
            for y, wt in g[x]:
                new_dis = dx + wt
                if new_dis < disappear[y] and (dis[y] < 0 or new_dis < dis[y]):
                    dis[y] = new_dis
                    heappush(h, (new_dis, y))
        return dis